In [10]:
from konlpy.tag import Twitter
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import normalize
from scipy.sparse import dok_matrix
from random import shuffle
import numpy as np
import json

twitter = Twitter()

In [11]:
with open("data/2017_03.json") as f:
    data = json.load(f)

In [12]:
data[0]['date']

'2017-03-01'

In [13]:
shuffle(data)
data = data[:5000]

In [14]:
titles = set()
articleList = []
articleNum = 0

for article in data:
    if article['title'] not in titles:
        # 기사 중복되지 않는 것들만 처리
        titles.add(article['title'])
        articleList.append(article['text'])
        articleNum += 1

del data

In [16]:
tags = set(['Noun', 'Verb', 'Adjective'])
spamWords = set([
    '하다', '되다', '이다', '전재', '배포', '무단', '기사', '금지', '사진', '보기', '뉴스', '있다',
    '돼다', '앵커', '위해', '라며', '그렇다', '보다', '되어다', '기자', '하는', '이런', '그런', '하는',
    '하였', '있었', '입니'
])
wordsList = []
index2voca = set()
for i, article in enumerate(articleList):
    if i % 1000 == 0:
        print(i)

    words = []
    p = twitter.pos(article, norm=True)
    for word, tag in p:
        if tag in tags and len(word) > 1 and word not in spamWords:
            words.append(word)
            index2voca.add(word)

    wordsList.append(words)

index2voca = list(index2voca)
voca2index = {w: i for i, w in enumerate(index2voca)}

print(len(index2voca))
print(len(wordsList))

0
1000
2000
3000
4000
36079
4936


In [17]:
# term-document matrix를 sparse matrix로 생성
tdm = dok_matrix((len(wordsList), len(index2voca)), dtype=np.float32)
print(tdm.shape)

for i, words in enumerate(wordsList):
    for word in words:
        tdm[i, voca2index[word]] += 1

tdm = tdm.tocsr()
# 각 document별로 l2-normalize
tdm = normalize(tdm)

(4936, 36079)


In [18]:
# LDA
K = 10
lda = LatentDirichletAllocation(n_components=K, learning_method='batch')
W = lda.fit_transform(tdm)
H = lda.components_

# 각 토픽별 키워드 출력
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(index2voca[index], end=" ")
    print("\n")

0th topic
대표 대선 민주당 후보 지사 경선 정당 바른 주자 의원 문재인 개헌 더불어 탈당 안희정 출마 국민의당 한국 토론회 김종인 

1th topic
증여 사전 상속세 증여세 지면 정보 테크 설계 일반 전략 아니 가지 이후 관심 입장 통한 절세 절감 납세 은퇴 

2th topic
켈리 방송사고 웃음 스타 부부 사탕 행복 등장 모터쇼 쏘나타 제네바 유럽 올해 라이즈 에서 출시 자동차 국내 변경 목표 

3th topic
시간 대선 지원 출마 보통사람 억원 정부 청춘 황교안 인상 소환 검찰 금리 주택 변경 공개 한국 문재인 웨스턴 출시 

4th topic
대구 경북 도시 찾아 시대 희망 세계 우리 낙동강 물의 시리즈 이야기 도심 유머 명소 버스 미래 강국 최고 기행 

5th topic
그래프 실시간 뜨는 발행 등록 청소년 인기 번호 사람 많은 접근하는 지금 얼마나 나타낸 시각 보호책임 편집 이상면 연월일 빌딩 

6th topic
결방 음악중심 반지 뷰스앤뉴스 점검 시스템 매일 정기 판타지 도봉 재방송 송죽동 허상 뮤직뱅크 편성표 김성재 슬램덩크 이스라엘 이미영 열병 

7th topic
최신 시신 관상동맥 무한도전 급사 민우 해안 침식 로꼬 소견 골절 심장질환 오대 동맥경화 거구 갈비뼈 머리뼈 솔비 김동리 국과수 

8th topic
접속 고백 당한 얼마 사기 많아 박시연 되었 다한 차단 귀하 하태 황기 하신 여기 주세 계속 클릭 읽으 배소영 

9th topic
대통령 탄핵 박근혜 서울 국민 헌재 심판 선고 집회 결정 헌법재판소 대한 인용 한국 오후 있는 대선 의원 수사 말했 



In [46]:
from sklearn.manifold import TSNE
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [50]:
# select random index
selectNum = 1000
randIndex = np.random.choice(W.shape[0], selectNum, replace=False)
randIndex.sort()

tsne = TSNE(n_components=2, init='pca', verbose=1)
W2d = tsne.fit_transform(W[randIndex, :])
topicIndex = [v.argmax() for v in W[randIndex, :]]


# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=800, plot_height=500, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in randIndex],
    'document': [articleList[randInd][:50] for randInd in randIndex],  # 해당 기사
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color',
    fill_alpha=0.7,
    line_alpha=0.7)

# 몇가지 interaction
p.legend.location = "top_left"
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.001s...
[t-SNE] Computed neighbors for 1000 samples in 0.038s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.330387
[t-SNE] Error after 1000 iterations: 0.665359


In [ ]:
# NMF
K = 10
nmf = NMF(n_components=K, max_iter=1000, alpha=0.1)
W = nmf.fit_transform(tdm)
H = nmf.components_

# 각 토픽별 키워드 출력
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(index2voca[index], end=" ")
    print("\n")

In [52]:
# select random index
selectNum = 1000
randIndex = np.random.choice(W.shape[0], selectNum, replace=False)
randIndex.sort()

tsne = TSNE(n_components=2, init='pca', verbose=1)
W2d = tsne.fit_transform(W[randIndex, :])
topicIndex = [v.argmax() for v in W[randIndex, :]]


# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=800, plot_height=500, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in randIndex],
    'document': [articleList[randInd][:50] for randInd in randIndex],  # 해당 기사
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color',
    fill_alpha=0.7,
    line_alpha=0.7)

# 몇가지 interaction
p.legend.location = "top_left"
hover = p.select(dict(type=HoverTool))
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.002s...
[t-SNE] Computed neighbors for 1000 samples in 0.089s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 64.169945
[t-SNE] Error after 1000 iterations: 0.688906
